In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile

In [2]:
path_to_zip = "Data/webmd.zip"
with zipfile.ZipFile(path_to_zip, 'r') as zip_ref:
    zip_ref.extractall("Data")

In [3]:
path_to_zip = "Data/drugsComTrain_raw.zip"
with zipfile.ZipFile(path_to_zip, 'r') as zip_ref:
    zip_ref.extractall("Data")

In [4]:
path_to_zip = "Data/drugsComTest_raw.zip"
with zipfile.ZipFile(path_to_zip, 'r') as zip_ref:
    zip_ref.extractall("Data")

<h2>The above unzip code in the 3 above cells extracts the zip into the Data directory. Only run them once in the beginning.
<h3>REMEMBER NOT TO PUSH THE EXTRACTED FILE TO GITHUB. IT IS TOO BIG.
    
<br>Explanation: The extracted .csv is around 168Mb. Github blocks pushes above a 100Mb. So get run this part of the code on your local machine so you can have the data to work with, but after you are done with it remember to delete the .csv files and only keep the .zip files when you push to github.

In [6]:
UCIdrug_train = pd.read_csv("Data/drugsComTrain_raw.csv", parse_dates=["date"])

In [ ]:
UCIdrug_test = pd.read_csv("Data/drugsComTest_raw.csv", parse_dates=["date"])

In [ ]:
webmd = pd.read_csv("Data/webmd.csv",parse_dates=["Date"])

Convert .csv data to pandas dataframe.

In [7]:
print("UCI Train shape :" ,UCIdrug_train.shape)
print("UCI Test shape :", UCIdrug_test.shape)
print("Webmd shape:", webmd.shape)

UCI Train shape : (161297, 7)
UCI Test shape : (53766, 7)
Webmd shape: (362806, 12)


Above we check the shape of the data just to check we're doing everything correct.<br>
As we can see the, UCI train data has 161297 rows and 7 columns.<br>
UCI test data has 53766 rows and 7 columns.<br>
Webmd data has 32806 rows, and 12 columns.

In [8]:
UCIdrug_train.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,2012-05-20,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,2010-04-27,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,2009-12-14,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,2015-11-03,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,2016-11-27,37


We print the first few rows of the UCI data to get an idea of what it's about. Arguably the most important information here is drugname, condition, review, rating, and possibly useful count.

In [9]:
webmd.head()

,Age,Condition,Date,Drug,DrugId,EaseofUse,Effectiveness,Reviews,Satisfaction,Sex,Sides,UsefulCount
0,75 or over,Stuffy Nose,2014-09-21,25dph-7.5peh,146724,5,5,I'm a retired physician and of all the meds I ...,5,Male,"Drowsiness, dizziness , dry mouth /nose/thro...",0
1,25-34,Cold Symptoms,2011-01-13,25dph-7.5peh,146724,5,5,cleared me right up even with my throat hurtin...,5,Female,"Drowsiness, dizziness , dry mouth /nose/thro...",1
2,65-74,Other,2012-07-16,warfarin (bulk) 100 % powder,144731,2,3,why did my PTINR go from a normal of 2.5 to ov...,3,Female,,0
3,75 or over,Other,2010-09-23,warfarin (bulk) 100 % powder,144731,2,2,FALLING AND DON'T REALISE IT,1,Female,,0
4,35-44,Other,2009-01-06,warfarin (bulk) 100 % powder,144731,1,1,My grandfather was prescribed this medication ...,1,Male,,1


Now we print the first few rows of the webmd data to get an idea of what it's about. Arguably the most important information here is drugname, condition, review, ease of use, satisfaction, and sides. However preproccesing is required, since the drug name also comes with the form of the drug, and a lot of sides(side effects) columns are empty.

In [61]:
num_unique_drug_webmd = webmd["Drug"].nunique() 
print("Unique drugs in webmd: ", num_unique_drug_webmd)
num_unique_drug_UCItrain = UCIdrug_train["drugName"].nunique() 
print("Unique drugs in UCI Train data: ", num_unique_drug_UCItrain)

Unique drugs in webmd:  7093
Unique drugs in UCI Train data:  3436


Above we get the unique values in the drug column in webmd data and drugname column in the UCI train data. However, we must consider that for the webmd data, there's the same drug but in different forms, which would show up as different values. We must take care of this to find the true number of unique drugs in the webmd dataset, to the best of our ability.

In [62]:
unique_drug_webmd = webmd["Drug"].unique() 
unique_drug_UCItrain = UCIdrug_train["drugName"].unique() 
print(type(unique_drug_UCItrain))
print(unique_drug_UCItrain[0:10])
print(unique_drug_webmd[0:10])

<class 'numpy.ndarray'>
['Valsartan' 'Guanfacine' 'Lybrel' 'Ortho Evra' 'Buprenorphine / naloxone'
 'Cialis' 'Levonorgestrel' 'Aripiprazole' 'Keppra'
 'Ethinyl estradiol / levonorgestrel']
['25dph-7.5peh' 'warfarin (bulk) 100 % powder' 'wymzya fe'
 '12 hour nasal relief spray, non-aerosol' 'pyrogallol crystals' 'lyza'
 'lysiplex plus liquid' 'lysteda' 'pyrithione zinc shampoo'
 'lysine acetate 4,000 mg oral powder packet']


As we see above, to the webmd dataset has drug names as the first word, while what comes after is a description of the form of the drug. This is what we must take care of. <br>
unique_drug_webmd contains the unique values from the drug column of webmdb. <br>
unique_drug_UCItrain contains the unique values from the drugname column of UCI train set.

In [103]:
sep = ' '
unique_drug_webmd_names = []
for i in unique_drug_webmd:
    unique_drug_webmd_names.append(i.split(sep, 1)[0])
    
unique_drug_webmd_names = np.array(unique_drug_webmd_names)
initial = len(unique_drug_webmd_names)

unique_drug_webmd_names = np.unique(unique_drug_webmd_names)   #returns sorted unique values in numpy array
print(unique_drug_webmd_names[1:10])
final = len(unique_drug_webmd_names)

print("\nLength of arrays with first word without removing repetition:",initial)
print("Length of arrays with first word without removing repetition:",final)
print("Repetitions removed:", initial - final)

['15dm-100gfn-5peh' '20dm-4cpm' '25dph-7.5peh' '4' '40pse-400gfn-20dm'
 '5-hydroxy-l-tryptophan' '5-hydroxytryptophan' '60pse-400gfn' '7-keto']

Length of arrays with first word without removing repetition: 7093
Length of arrays with first word without removing repetition: 4615
Repetitions removed: 2478


The above cell keeps only the first word in the unique_drug_webmd array to the unique_drug_webmd_names array. We have to check for repeats in unique_drug_webmd_names. Then, we obtain the unique elements in the unique_drug_webmd_names array. We check the new length of array and it is 4615, and we seem to have removed 2478 repetions.

In [64]:
unique_drug_UCItrain_upper = [word.upper() for word in unique_drug_UCItest]

unique_drug_webmd_upper = [word.upper() for word in unique_drug_webmd_names]
    
print(unique_drug_UCItrain_upper[0:5])
print(unique_drug_webmd_names[0:5])

['A / B OTIC', 'ABACAVIR / DOLUTEGRAVIR / LAMIVUDINE', 'ABACAVIR / LAMIVUDINE', 'ABATACEPT', 'ABILIFY']
['12' '15dm-100gfn-5peh' '20dm-4cpm' '25dph-7.5peh' '4']


Now we want to do a comparision, to find if there are any drugs in both datasets. For this, we do not want any conflicts in the letters being uppercase or lowercase, and so we convert everything in both arrays to uppercase.

In [65]:
match_count = 0
common_drugs = []
for i in unique_drug_UCItrain_upper:
    if i in unique_drug_webmd_names:
        match_count+=1
        common_drugs.append(i)

print(match_count)
print(common_drugs[0:10])

0
[]


We have 1570 drugs that are present in both UCIdrugs train data and the webmd dataset. Few of the common drugs are printed above.

<h2> For Zhiyun and Kevin.

In [66]:
UCIdrug_train.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,2012-05-20,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,2010-04-27,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,2009-12-14,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,2015-11-03,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,2016-11-27,37


In [67]:
UCIdrug_train_RS = UCIdrug_train.drop(['uniqueID','date'],axis=1)
UCIdrug_train_RS.head()

,drugName,condition,review,rating,usefulCount
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,27
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,192
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,17
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,10
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,37


In [68]:
UCIdrug_test_RS = UCIdrug_test.drop(['uniqueID','date'],axis=1)
UCIdrug_test_RS.head()

,drugName,condition,review,rating,usefulCount
0,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10,22
1,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8,17
2,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9,3
3,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9,35
4,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9,4


In [69]:
webmd_RS = webmd.drop(['Date'],axis=1)
webmd_RS.head()

,Age,Condition,Drug,DrugId,EaseofUse,Effectiveness,Reviews,Satisfaction,Sex,Sides,UsefulCount
0,75 or over,Stuffy Nose,25dph-7.5peh,146724,5,5,I'm a retired physician and of all the meds I ...,5,Male,"Drowsiness, dizziness , dry mouth /nose/thro...",0
1,25-34,Cold Symptoms,25dph-7.5peh,146724,5,5,cleared me right up even with my throat hurtin...,5,Female,"Drowsiness, dizziness , dry mouth /nose/thro...",1
2,65-74,Other,warfarin (bulk) 100 % powder,144731,2,3,why did my PTINR go from a normal of 2.5 to ov...,3,Female,,0
3,75 or over,Other,warfarin (bulk) 100 % powder,144731,2,2,FALLING AND DON'T REALISE IT,1,Female,,0
4,35-44,Other,warfarin (bulk) 100 % powder,144731,1,1,My grandfather was prescribed this medication ...,1,Male,,1


Removed the useless values for recommender system. RS = Recommender System.

In [70]:
unique_drug_UCItest = UCIdrug_test['drugName'].unique()
print(unique_drug_UCItest)

['Mirtazapine' 'Mesalamine' 'Bactrim' ... 'Guarana' 'Maprotiline'
 'FluMist']


In [71]:
unique_drug_UCItest = np.unique(unique_drug_UCItest)
unique_drug_UCItest_upper = [word.upper() for word in unique_drug_UCItest]
countmiss = 0

for i in unique_drug_UCItest_upper:
    if i not in unique_drug_UCItrain_upper:
        countmiss+=1

print(countmiss)

0


So what the above cell does, is it sorts all the unique names of the drugs in the three datasets alphabetically. Countmiss shows the number of unique values in UCItest that are not present in UCItrain.

In [111]:
UCIdrug_train_RS.head()

,drugName,condition,review,rating,usefulCount
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,27
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,192
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,17
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,10
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,37


In [198]:
drug_ids = {}
idtrk = 0
for i in unique_drug_UCItrain:
    drug_ids[i] = idtrk
    idtrk+=1
    
#print(drug_ids)
print(len(drug_ids))

drugid_vals = drug_ids.keys()
drugid_vals = [word.upper() for word in drugid_vals]

3436


Above we start the process of assigning a unique id to each drug in our datasets. Right now, we put every unique value from the UCITrain column into a disctionary drug_ids. The key is just a counter that starts from 0.

In [261]:
for i in unique_drug_webmd:
    temp = i.split()
    dummy_c = 0
    for j in temp:
        if j.upper() in drugid_vals:
            c = 1
    if c == 0:
        drug_ids[i] = idtrk
        idtrk+=1
    
    c = 0

Here we go through the webmd dataset, but since the webmd dataset contains descriptions of drugs, not necessarily just the names, we have to split each item in that data and check if any of the words are already in out drug_ids dictionary. If yes, we don't add them to the dictionary, if not, we do.

In [262]:
drugcol = list(drug_ids.keys())
indexer = [i for i in range(len(drugcol))]
drugidtodf = {'Drugname':drugcol,'id':indexer}

drugid_df = pd.DataFrame.from_dict(drugidtodf)
print(drugid_df.head())

                   Drugname  id
0                 Valsartan   0
1                Guanfacine   1
2                    Lybrel   2
3                Ortho Evra   3
4  Buprenorphine / naloxone   4


Created a new dataframe containing drugids and drugnames.

In [265]:
drugid_df.to_csv("Data\drugid_df.csv")

In [219]:
conditions_UCItrain = np.array(UCIdrug_train_RS['condition'].unique())
#print(conditions_UCItrain)
to_remove = []
for i in range(len(conditions_UCItrain)-1):
    temp = str(conditions_UCItrain[i])
    temp = temp.split()
    if "comment" in temp:
        print(conditions_UCItrain[i])
        to_remove.append(i)
        
#print(len(conditions_UCItrain))
#print(to_remove)

conditions_UCItrain_unique = np.delete(conditions_UCItrain,to_remove) 
#print(conditions_UCItrain_unique)

2</span> users found this comment helpful.
4</span> users found this comment helpful.
3</span> users found this comment helpful.
11</span> users found this comment helpful.
0</span> users found this comment helpful.
1</span> users found this comment helpful.
142</span> users found this comment helpful.
8</span> users found this comment helpful.
13</span> users found this comment helpful.
6</span> users found this comment helpful.
94</span> users found this comment helpful.
28</span> users found this comment helpful.
16</span> users found this comment helpful.
27</span> users found this comment helpful.
75</span> users found this comment helpful.
15</span> users found this comment helpful.
12</span> users found this comment helpful.
9</span> users found this comment helpful.
7</span> users found this comment helpful.
35</span> users found this comment helpful.
54</span> users found this comment helpful.
17</span> users found this comment helpful.
79</span> users found this comment helpf

As you can see in the above cells, these values were found in the condition column. These are obviously not conditions, so I'm removing them.

In [240]:
conditions_Webmd_unique = np.array(webmd_RS['Condition'].unique())
print(conditions_Webmd_unique)
print(len(conditions_Webmd))

['Stuffy Nose' 'Cold Symptoms' 'Other' ...
 'Combative and Explosive Behavior' 'Lead Poisoning'
 'Poisoning from Swallowed Unknown Substance']
1806


Checked for comments in the conditions of webmd, weren't any.

In [250]:
condition_ids = {}
idtrk = idtrk + 134452
for i in conditions_UCItrain_unique:
    condition_ids[i] = idtrk
    idtrk+=1
    
print(len(condition_ids))
#print(condition_ids)

checker_cond = condition_ids.keys()
for i in checker_cond:
    i = str(i)
    i.upper()

812


Checker_cond is only here to ensure no repeating values.

In [258]:
for i in conditions_Webmd_unique:
    if str(i).upper() not in checker_cond:
        condition_ids[i] = idtrk
        idtrk+=1
        
print(len(conditions_Webmd_unique))

1806


In [259]:
conditioncol = list(condition_ids.keys())
indexer = list(condition_ids.values())
conditionidtodf = {'Condition':conditioncol,'id':indexer}

conditionid_df = pd.DataFrame.from_dict(conditionidtodf)
print(conditionid_df.head())

                      Condition      id
0  Left Ventricular Dysfunction  137070
1                          ADHD  137071
2                 Birth Control  139691
3             Opiate Dependence  137073
4  Benign Prostatic Hyperplasia  137074


In [267]:
conditionid_df.to_csv("Data\conditionid_df.csv")

<h3> RUN THIS CODE AFTER YOU'RE DONE WORKING. IT WILL REMOVE THE BIG .CSV FILES THAT CANNOT BE PUSHED TO GITHUB.

In [268]:
import os
os.remove("Data\drugsComTest_raw.csv")
os.remove("Data\drugsComTrain_raw.csv")
os.remove("Data\webmd.csv")